In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tuner import HyperparameterTuner

hpo_training_job = 'ENTER_HPO_JOBNAME_HERE'

assert(hpo_training_job != 'ENTER_HPO_JOBNAME_HERE')

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

tuner = HyperparameterTuner.attach(hpo_training_job, sagemaker_session=sagemaker_session)

In [ ]:
predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Choose some data and use it for a prediction <a class="anchor" id="prediction_request"></a>

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [ ]:
import itertools
import pandas as pd

raw_data = pd.read_csv('s3://sagemaker-workshop/2019-01-22/data/iris.csv', header=None, engine="python") 

test_X = raw_data.iloc[:,1:]
test_y = raw_data.iloc[:,0]

Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The output from the endpoint return an numerical representation of the classification prediction; in the original dataset, these are flower names, but in this example the labels are numerical. We can compare against the original label that we parsed.

In [ ]:
test_X.values

In [ ]:
print(predictor.predict(test_X.values))
print(test_y.values)

### Endpoint cleanup <a class="anchor" id="endpoint_cleanup"></a>

When you're done with the endpoint, you'll want to clean it up.

In [ ]:
predictor.delete_endpoint()